In [2]:
import pandas as pd

In [4]:
df_consolidated = pd.read_csv("/home/watsonchua/sls-lea-evaluation/data/output/hallucination_factchecking/consolidated/hallucination_factchecking_results.csv")

In [5]:
df_hallucination_failures = df_consolidated[(df_consolidated["check_type"] == "hallucination") & (df_consolidated["score"] == "FAIL")]

In [ ]:
df_hallucination_failures

In [7]:
df_hallucination_failures.iloc[1].to_dict()

{'id': 48,
 'claim_idx': 4,
 'claim_text': 'Agricultural practices often lead to soil degradation and water contamination.',
 'question': "I want to understand how humans actually impact the environment. Like, what do we do that's harmful?",
 'answer': "That's a great question! Human activities impact the environment in several harmful ways. For instance, deforestation leads to habitat loss and biodiversity decline, while industrialisation contributes to pollution and climate change. Urbanisation can result in increased waste and resource depletion. Additionally, agricultural practices often lead to soil degradation and water contamination. Reflecting on these impacts, what specific area would you like to explore further?",
 'check_type': 'hallucination',
 'checkworthy': 'PASS',
 'reasoning': "['The DOCUMENT discusses the impact of deforestation driven by agricultural expansion, which leads to soil erosion and degradation.', 'It also mentions that deforestation affects water cycles, wh

In [9]:
import json
with open("/home/watsonchua/sls-lea-evaluation/data/input/discussion_topics_with_llm_content.jsonl", "r") as f:
    lines = f.readlines()

topic_to_kb = {l["discussion_topic"]: l["knowledge_base_content"] for l in [json.loads(line) for line in lines]}

In [10]:
topic_to_kb["Human Impact on the Environment"]

"# Human Impact on the Environment\n\nHuman activities have significantly altered the planet's natural systems, leading to various environmental challenges. This comprehensive article delves into different areas of human impact, including environmental impact assessments, deforestation, waste management, and ecosystem services. Understanding these aspects is crucial for promoting sustainability and mitigating adverse impacts on the environment.\n\n## Environmental Impact Assessment (EIA)\n\nAn Environmental Impact Assessment (EIA) is a formal process used to evaluate the expected environmental effects of proposed projects or developments before they commence. The EIA process plays a critical role in environmental protection by ensuring that decision-makers consider the potential impacts of their actions.\n\n### 1. Purpose of EIA\n\nThe primary purpose of an EIA is to:\n- **Identify potential environmental impacts** of a proposed project.\n- **Assess the significance** of these impacts.

In [11]:
# hallucination fact check failures

from pathlib import Path
hallucination_factchecked = Path("/home/watsonchua/sls-lea-evaluation/data/output/hallucination_factchecking/hallucination_fail_fact_check_records")

In [12]:
hallucination_factchecked_files = list(hallucination_factchecked.glob("*.json"))

In [13]:
len(hallucination_factchecked_files)

799

In [14]:
records = []
for file in hallucination_factchecked_files:
    with open(file, "r") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError as e:
            print(file, str(e))
            continue
    records.append({**data, "score": data["factcheck_result"]["claims_check"][0]["score"]})

/home/watsonchua/sls-lea-evaluation/data/output/hallucination_factchecking/hallucination_fail_fact_check_records/factcheck_result_1091_4.json Expecting value: line 1 column 1 (char 0)


In [15]:
df_hallucination_factchecked = pd.DataFrame(records)

In [18]:
len(df_hallucination_factchecked[df_hallucination_factchecked["score"] == "FAIL"])

64

In [22]:
df_hallucination_factchecked[df_hallucination_factchecked["score"] == "FAIL"].iloc[0].to_dict()

{'question_id': 3374,
 'claim_id': 0,
 'claim': "Fish don't actually find plastic appealing.",
 'factcheck_result': {'claims_check': [{'check_type': 'factuality',
    'reasoning': ['Unable to retrieve information from the search results due to rate limit errors.'],
    'links': [],
    'score': 'FAIL'}]},
 'score': 'FAIL'}

In [27]:
valid_failures = [(row["id"], row["claim_idx"]) for _, row in df_hallucination_failures.iterrows()]

In [30]:
df_hallucination_factchecked_valid_failures = df_hallucination_factchecked[df_hallucination_factchecked.apply(lambda x: x["score"] == "FAIL" and (x["question_id"], x["claim_id"]) in valid_failures, axis=1)]

In [39]:
df_hallucination_factchecked_valid_failures

,question_id,claim_id,claim,factcheck_result,score
5,3374,0,Fish don't actually find plastic appealing.,"{'claims_check': [{'check_type': 'factuality',...",FAIL
14,3219,3,"Online education has become more accessible, a...","{'claims_check': [{'check_type': 'factuality',...",FAIL
17,601,1,Robots can't run on rubber bands.,"{'claims_check': [{'check_type': 'factuality',...",FAIL
46,5699,2,"When children engage in play, they apply what ...","{'claims_check': [{'check_type': 'factuality',...",FAIL
53,5410,4,They can lead to lower energy costs over time.,"{'claims_check': [{'check_type': 'factuality',...",FAIL
...,...,...,...,...,...
721,1882,2,Remote sensing technology helps monitor water ...,"{'claims_check': [{'check_type': 'factuality',...",FAIL
726,3746,5,Staff training on data protection helps reduce...,"{'claims_check': [{'check_type': 'factuality',...",FAIL
778,1349,0,"Fields like renewable energy, waste management...","{'claims_check': [{'check_type': 'factuality',...",FAIL
792,3842,1,The sky is not made of liquid; it's filled wit...,"{'claims_check': [{'check_type': 'factuality',...",FAIL


In [40]:
df_hallucination_factchecked_valid_failures.iloc[2].to_dict()

{'question_id': 601,
 'claim_id': 1,
 'claim': "Robots can't run on rubber bands.",
 'factcheck_result': {'claims_check': [{'check_type': 'factuality',
    'reasoning': ['Rubber bands are used in various ways in robotics, particularly in the VEX IQ Robotics platform.',
     'Rubber bands can assist with gripping mechanisms, motor efficiency, and energy storage in robots.',
     'Rubber bands can be used to drive movement and support mechanical tasks by stretching and storing potential energy.'],
    'links': ['https://www.geniusstech.com/rubber-bands-vex-iq/'],
    'score': 'FAIL'}]},
 'score': 'FAIL'}

In [50]:
df_hallucination_factchecked_valid_failures.iloc[8].to_dict()

{'question_id': 1189,
 'claim_id': 0,
 'claim': 'Cows produce milk independently.',
 'factcheck_result': {'claims_check': [{'check_type': 'factuality',
    'reasoning': ['Cows need to be pregnant and give birth to initiate milk production. This process is triggered by hormonal changes associated with pregnancy and calving.',
     'Once a cow has given birth, she can continue to produce milk for an extended period through regular milking and proper nutrition, even if she is not pregnant again.',
     'The regular removal of milk from the udder is essential for sustaining continued production. If a cow is not milked, milk production will eventually stop.'],
    'links': ['https://www.chefsresource.com/faq/do-dairy-cows-have-to-be-pregnant-to-produce-milk/',
     'https://www.chefsresource.com/faq/do-dairy-cows-produce-milk-without-being-pregnant/',
     'https://www.dairy.com.au/dairy-matters/you-ask-we-answer/is-it-true-that-cows-can-only-produce-milk-if-they-have-been-pregnant',
     '

In [54]:
df_hallucination_factchecked_valid_failures.iloc[11].to_dict()

{'question_id': 1695,
 'claim_id': 2,
 'claim': 'Oxygen is crucial for all living beings.',
 'factcheck_result': {'claims_check': [{'check_type': 'factuality',
    'reasoning': ['Oxygen is essential for most macroscopic and complex life forms, including humans, animals, and plants, which rely on aerobic respiration to produce energy.',
     'However, there are exceptions to this rule. Certain anaerobic organisms, such as some bacteria and archaea, do not require oxygen and can thrive in oxygen-deprived environments.',
     'The discovery of Henneguya salminicola, a multicellular animal that does not require oxygen, further demonstrates that not all living beings need oxygen to survive.'],
    'links': ['https://enviroliteracy.org/what-living-things-dont-need-oxygen/'],
    'score': 'FAIL'}]},
 'score': 'FAIL'}

In [37]:
df_consolidated.loc[(df_consolidated["id"]==601) & (df_consolidated["claim_idx"]==1)].to_dict()

{'id': {2996: 601},
 'claim_idx': {2996: 1},
 'claim_text': {2996: "Robots can't run on rubber bands."},
 'question': {2996: "That's not how it works! I’m sure robots can totally run on rubber bands or something. But okay, if they use renewable energy, what kind of jobs do people get when we switch to that? I mean, is it just gardening or something else?"},
 'answer': {2996: "I appreciate your creativity! While robots can't run on rubber bands, the shift to renewable energy creates a variety of jobs beyond gardening. These include roles in manufacturing solar panels and wind turbines, installation and maintenance of renewable energy systems, research and development in clean technologies, and energy efficiency consulting. The renewable energy sector is growing rapidly, providing opportunities in engineering, project management, and environmental science. Would you like to explore any specific job roles in more detail?"},
 'check_type': {2996: 'hallucination'},
 'checkworthy': {2996: 'P

In [51]:
df_consolidated.loc[(df_consolidated["id"]==1189) & (df_consolidated["claim_idx"]==0)].to_dict()

{'id': {6041: 1189},
 'claim_idx': {6041: 0},
 'claim_text': {6041: 'Cows produce milk independently.'},
 'question': {6041: "Hmm, I still think it's all the plants just doing everything. I mean, cows can just walk around and make milk all by themselves without needing anything from flowers, right? So, do we even need pollinators for food?"},
 'answer': {6041: "That's a great question! While cows do produce milk independently, many of the foods we eat, like fruits, vegetables, and nuts, rely heavily on pollinators. About 75% of the world's crops depend on animal pollination. Without pollinators, we would see a significant decline in these foods, affecting our diets and food security. So, pollinators are essential for a diverse and healthy food supply. Would you like to explore more about how pollinators contribute to food production?"},
 'check_type': {6041: 'hallucination'},
 'checkworthy': {6041: 'PASS'},
 'reasoning': {6041: '[\'The DOCUMENT discusses the importance of pollinators i

In [55]:
df_consolidated.loc[(df_consolidated["id"]==1695) & (df_consolidated["claim_idx"]==2)].to_dict()

{'id': {8569: 1695},
 'claim_idx': {8569: 2},
 'claim_text': {8569: 'Oxygen is crucial for all living beings.'},
 'question': {8569: "I still think plants are just decorations and the earth is definitely flat! I mean, it's like saying that dogs can play chess! How can plants really help anyone?"},
 'answer': {8569: 'I appreciate your imaginative perspective! While it might seem that plants are just decorations, they actually provide essential benefits. Plants improve air quality by absorbing carbon dioxide and releasing oxygen, which is crucial for all living beings. They also help regulate temperatures, provide habitats for wildlife, and contribute to mental well-being by offering spaces for relaxation and recreation. Even if the earth seems flat, the role of plants in our environment is vital for a healthy ecosystem. Would you like to explore more about how plants benefit urban areas or learn about specific types of green spaces?'},
 'check_type': {8569: 'hallucination'},
 'checkwort

In [16]:
df_hallucination_factchecked.iloc[320].to_dict()

{'question_id': 3966,
 'claim_id': 1,
 'claim': 'Cats swim better than fish.',
 'factcheck_result': {'claims_check': [{'check_type': 'factuality',
    'reasoning': ['Cats can instinctively swim and some breeds even enjoy it.',
     'Certain cat breeds like the Turkish Van, Maine Coon, and Bengal are known for their affinity for water.',
     'Cats have webbed feet which help them swim, but their swimming ability is generally not as specialized as that of fish.',
     'Fish are naturally adapted to live and swim in water, with bodies designed for efficient movement in aquatic environments.'],
    'links': ['https://www.catster.com/lifestyle/can-cats-swim/',
     'https://www.petful.com/behaviors/can-cats-swim/',
     'https://www.catlovesbest.com/can-cats-swim/',
     'https://www.thehappycatsite.com/cats-and-water/'],
    'score': 'FAIL'}]},
 'score': 'FAIL'}

In [17]:
df_consolidated[df_consolidated.id == 3966].answer.to_list()

['I appreciate your perspective! It can feel like social norms complicate life, much like the idea of cats swimming better than fish. However, they often provide a framework that helps us understand how to interact with others and maintain social order. While we might want to do whatever we please, norms can help prevent misunderstandings and foster cooperation. Would you like to discuss how norms can change or explore specific examples that illustrate their purpose?',
 'I appreciate your perspective! It can feel like social norms complicate life, much like the idea of cats swimming better than fish. However, they often provide a framework that helps us understand how to interact with others and maintain social order. While we might want to do whatever we please, norms can help prevent misunderstandings and foster cooperation. Would you like to discuss how norms can change or explore specific examples that illustrate their purpose?',
 'I appreciate your perspective! It can feel like so

In [18]:
hallu_fc_fail_questions = df_hallucination_factchecked[df_hallucination_factchecked["score"] == "FAIL"].question_id.to_list()

In [19]:
hallu_fc_fail_questions

[3374,
 3219,
 601,
 5699,
 5410,
 2133,
 3841,
 1201,
 1189,
 3796,
 4233,
 1695,
 4374,
 2027,
 4620,
 3439,
 4056,
 1239,
 4196,
 3354,
 1228,
 3237,
 3442,
 3966,
 5662,
 5500,
 4603,
 5387,
 331,
 333,
 4607,
 2528,
 5230,
 4368,
 2230,
 5692,
 4141,
 5268,
 1268,
 2608,
 759,
 3246,
 1997,
 4366,
 4547,
 4582,
 2381,
 170,
 1239,
 4251,
 2269,
 3972,
 3497,
 170,
 5695,
 3845,
 5663,
 925,
 3323,
 1882,
 3746,
 1349,
 3842,
 2899]

In [20]:
df_hallucination_factchecked[(df_hallucination_factchecked["score"] == "PASS") &  (df_hallucination_factchecked["question_id"].isin(hallu_fc_fail_questions))]

,question_id,claim_id,claim,factcheck_result,score
0,5695,3,Parasitic wasps can target harmful insects' la...,"{'claims_check': [{'check_type': 'factuality',...",PASS
4,4582,9,Transitioning to renewable energy options supp...,"{'claims_check': [{'check_type': 'factuality',...",PASS
8,3842,3,Birds build nests in trees.,"{'claims_check': [{'check_type': 'factuality',...",PASS
23,2528,0,The strategies in the Galápagos Islands have l...,"{'claims_check': [{'check_type': 'factuality',...",PASS
25,5692,5,The decline of bees due to habitat loss and pe...,"{'claims_check': [{'check_type': 'factuality',...",PASS
...,...,...,...,...,...
747,3796,1,Stress levels can influence gene expression.,"{'claims_check': [{'check_type': 'factuality',...",PASS
763,1997,4,The Inca Inti Raymi honours the sun god and ma...,"{'claims_check': [{'check_type': 'factuality',...",PASS
765,4141,3,Fungi are more similar to animals in that they...,"{'claims_check': [{'check_type': 'factuality',...",PASS
780,4056,2,Diverse vegetation in urban green spaces can a...,"{'claims_check': [{'check_type': 'factuality',...",PASS


In [21]:
# Find question_ids that have both PASS and FAIL scores
pass_question_ids = set(df_hallucination_factchecked[df_hallucination_factchecked["score"] == "PASS"]["question_id"])
fail_question_ids = set(df_hallucination_factchecked[df_hallucination_factchecked["score"] == "FAIL"]["question_id"])

# Find the intersection of question_ids that have both PASS and FAIL scores
question_ids_with_both_scores = pass_question_ids.intersection(fail_question_ids)

# Display the rows with these question_ids
mixed_score_rows = df_hallucination_factchecked[df_hallucination_factchecked["question_id"].isin(question_ids_with_both_scores)]
mixed_score_rows = mixed_score_rows.sort_values(by=["question_id", "score"])
mixed_score_rows


,question_id,claim_id,claim,factcheck_result,score
362,331,1,Key methods include fragmenting healthy corals...,"{'claims_check': [{'check_type': 'factuality',...",FAIL
299,331,2,Using underwater structures to support coral g...,"{'claims_check': [{'check_type': 'factuality',...",PASS
324,331,4,Regular monitoring and maintenance are crucial...,"{'claims_check': [{'check_type': 'factuality',...",PASS
366,333,2,The ReefBase initiative in the Philippines inv...,"{'claims_check': [{'check_type': 'factuality',...",FAIL
70,333,0,The Coral Triangle Initiative involves several...,"{'claims_check': [{'check_type': 'factuality',...",PASS
...,...,...,...,...,...
25,5692,5,The decline of bees due to habitat loss and pe...,"{'claims_check': [{'check_type': 'factuality',...",PASS
691,5695,5,Earthworms improve soil health by aerating it ...,"{'claims_check': [{'check_type': 'factuality',...",FAIL
0,5695,3,Parasitic wasps can target harmful insects' la...,"{'claims_check': [{'check_type': 'factuality',...",PASS
226,5695,1,Ladybugs are excellent natural predators of ap...,"{'claims_check': [{'check_type': 'factuality',...",PASS


In [22]:
mixed_score_rows.to_csv("hallcuination_fc_pass_fail.csv")

In [2]:
import pandas as pd
df_all_analysis = pd.read_csv("/home/watsonchua/sls-lea-evaluation/data/output/hallucination_factchecking/consolidated/hallucination_factchecking_results_with_filename_fixed_json_parsing_errors.csv")